<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E6%A0%AA%E4%BE%A1%E5%88%86%E6%9E%90%E9%96%A2%E6%95%B0_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=da319250eca6d3dc90ab3d0b32079b3bb9c5104491507937168e00d23171fc54
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [1]:
import pandas_datareader.data as pdr

# 株価データを取得するメソッド
def get_stock_data(code):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :return: 株価データのデータフレーム
    """
    df = pdr.DataReader(f'{code}.JP', 'stooq').sort_index()
    return df

In [59]:
import plotly.graph_objs as go
import datetime as dt
import pandas as pd
import numpy as np
import pandas_ta as ta

def show_plotly(code, name, start, end=dt.date.today(), sma=[], bbands={}, gcdc=False, title=''):
    """ローソク足や移動平均線のチャートを表示する

    :param code: 取得する株価データの銘柄コード
    :param code: 取得する株価データの銘柄名
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    :param sma: チャートに移動平均線を表示するためのリスト（例: [5, 25, 75]）
    :param bbands: チャートにボリンジャーバンドを表示するための辞書（例: {'sma': 25, 'bbands': [1, 2]} or {'sma': 25, 'fill': True}）
    :param gcdc: チャートの移動平均線のゴールデンクロスとデッドクロスを表示するかどうか
    :param title: チャートのタイトルを表示
    """
    # 株価データの取得
    df = get_stock_data(code)
    close = df['Close']

    # ▼▼▼▼▼テクニカル指標の算出▼▼▼▼▼

    # ①移動平均線
    for ma in sma:
        df[f'ma{ma}'] = ta.sma(close, ma)
    
    # ①'ゴールデンクロスとデッドクロス
    if gcdc and len(sma) >= 2:
        cross = df[f'ma{sma[0]}'] > df[f'ma{sma[1]}']
        cross_shift = cross.shift(1)
        temp_gc = (cross != cross_shift) & (cross == True)
        temp_dc = (cross != cross_shift) & (cross == False)
        df['gc'] = [m if g == True else np.nan for g, m in zip(temp_gc, df[f'ma{sma[0]}'])]
        df['dc'] = [m if d == True else np.nan for d, m in zip(temp_dc, df[f'ma{sma[1]}'])]
    
    # ②ボリンジャーバンド
    if bbands:
        if not sma:
            df[f'ma{bbands["sma"]}'] = ta.sma(close, bbands['sma'])
        # 辞書bbandsのキーに'fill'が存在し、値がTrueかどうか確認
        if 'fill' not in bbands or not bbands['fill']:
            for n in bbands['bbands']:
                bbands_df = ta.bbands(close, length=bbands["sma"], std=n)
                df[f'lower{n}'], df[f'upper{n}'] = bbands_df.iloc[:, 0], bbands_df.iloc[:, 2]
        else:
            bbands_df = ta.bbands(close, length=bbands["sma"], std=2)
            df['lower2'], df['upper2'] = bbands_df.iloc[:, 0], bbands_df.iloc[:, 2]

    # ▲▲▲▲▲テクニカル指標の算出▲▲▲▲▲
    
    # 日付で絞り込み
    cdf = df[start:end]

    # インデックスを文字列型に変更
    cdf.index = pd.to_datetime(cdf.index).strftime('%Y/%m/%d')

    # データ定義（ローソク足のデータ作成）
    data = [
        go.Candlestick(x=cdf.index, open=cdf['Open'], high=cdf['High'], low=cdf['Low'], close=cdf['Close'], name='ローソク足',
                       increasing_line_color='green',
                       increasing_line_width=1.0,
                       increasing_fillcolor='green',
                       decreasing_line_color='red',
                       decreasing_line_width=1.0,
                       decreasing_fillcolor='red')
    ]

    # チャートに表示するために使用する変数
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    icolor = 0  # panel=0のcolorのインデックスを表す

    # ▼▼▼▼▼テクニカル指標のチャートへの表示▼▼▼▼▼

    # ①移動平均線
    for ma in sma:
        data.append(go.Scatter(x=cdf.index, y=cdf[f'ma{ma}'], name=f'MA({ma})', line={'color': colors[icolor], 'width': 1.2}))
        icolor += 1
    
    # ①'ゴールデンクロスとデッドクロス
    if gcdc and len(sma) >= 2:
        data.append(go.Scatter(x=cdf.index, y=cdf['gc'], name='ゴールデンクロス', opacity=0.5, mode='markers', marker={'size': 15, 'color': 'purple'}))
        data.append(go.Scatter(x=cdf.index, y=cdf['dc'], name='デッドクロス', opacity=0.5, mode='markers', marker={'size': 15, 'color': 'black', 'symbol': 'x'}))
    
    # ②ボリンジャーバンド
    bbands_fill_between = {}
    if bbands:
        if not sma:
            data.append(go.Scatter(x=cdf.index, y=cdf[f'ma{bbands["sma"]}'], name=f'MA({bbands["sma"]})', line={'color': colors[icolor], 'width': 1.2}))
            icolor += 1
        # 辞書bbandsのキーに'fill'が存在し、値がTrueかどうか確認
        if 'fill' not in bbands or not bbands['fill']:
            for n in bbands['bbands']:
                data.append(go.Scatter(x=cdf.index, y=cdf[f'upper{n}'], name=f'+{n}σ', line={'color': colors[icolor], 'width': 1.2}))
                data.append(go.Scatter(x=cdf.index, y=cdf[f'lower{n}'], name=f'−{n}σ', line={'color': colors[icolor], 'width': 1.2}))
                icolor += 1
        else:
            data.append(go.Scatter(x=cdf.index, y=cdf['upper2'], name='', line={'color': 'lavender', 'width': 0}))
            data.append(go.Scatter(x=cdf.index, y=cdf['lower2'], name=f'±2σ', line={'color': 'lavender', 'width': 0}, fill='tonexty', fillcolor='rgba(170, 170, 170, .2)'))
            icolor += 1
    
    # ▲▲▲▲▲テクニカル指標のチャートへの表示▲▲▲▲▲

    # チャートのタイトルを作成
    title_text = f'{code} {name}'
    title_text = f'{title_text} | {title}' if title else title_text

    # レイアウト定義
    layout = {
        'title': {'text': title_text, 'x': 0.5},
        'xaxis': {'title': '日付', 'rangeslider': {'visible': False}},
        'yaxis': {'title': '価格（円）', 'side': 'left', 'tickformat': ','},
        'plot_bgcolor': 'light blue'
    }

    # グラフ生成
    fig = go.Figure(data=data, layout=go.Layout(layout))

    # レイアウトを更新
    fig.update_layout({
        'xaxis': {
            # 日付を3日おきに表示するように設定
            'tickvals': cdf.index[::3],
        }
    })

    # 表示
    fig.show()

In [60]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True)

In [62]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], bbands={'sma': 25, 'fill': True})